In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler, Stream, API

import time

import pandas as pd

In [2]:
access_token = "852009551876431872-OfvYX17eqrPz9eERGaRVxKfkBPVALyO"
access_token_secret = "koQa3hgW22EsgdvseQVsj3KnYbzHc564xEVfr7lYiPGhy"
consumer_key = "95fyXonGGIHKgfothfbOOAM7p"
consumer_secret = "6KWDuC87go4CbFE6jLdRnHWGFcj2Fl9hQvdizfaiwCOdZliv49"

In [126]:
class MyStreamListener(StreamListener):

    def __init__(self):
        self.api = API()
        self.df = pd.DataFrame(columns=['ID', 'Tweet', 'Time', 'User', 'Likes', 'Replies', 'Retweets', 'in_response_to', 'response_type', 'debug'])
        self.userData = pd.DataFrame(columns=['username', 'created', 'location', 'has_location','is_verified', 'total_tweets', 'total_following', 'total_followers', 'total_likes', 'has_avatar', 'has_background', 'is_protected', 'profile_modified'])

    def on_status(self, tweet):
        response_type = None
        in_response_to = tweet.in_reply_to_status_id
         
        if (in_response_to == None):
            if hasattr(tweet, 'retweeted_status'):
                response_type = 'retweet'
                in_response_to = tweet.retweeted_status.id
            elif hasattr(tweet, 'quoted_status'):
                response_type = 'quoted_retweet'
                in_response_to = tweet.quoted_status.id
        else:
            response_type = 'reply'
        
        self.df = self.df.append(pd.Series({'ID': tweet.id, 'Tweet': tweet.text, 'Time': tweet.created_at, 'User': tweet.user.screen_name, 'Likes': tweet.favorite_count, 'Replies': 0,'Retweets': tweet.retweet_count, 'in_response_to': in_response_to, 'response_type': response_type, 'debug': tweet}), ignore_index=True)
        
        has_avatar = 0 if 'default_profile_images' in tweet.user.profile_image_url else 1
        has_background = int(tweet.user.profile_use_background_image)
        has_location = int(tweet.user.geo_enabled)
        
        if (has_location == 1):
            location = tweet.user.location
        else:
            location = 0

        profile_modified = 1 if has_background == 1 or has_avatar == 1 or has_location != 0 else 0
        
        self.userData = self.userData.append(pd.Series({'username':tweet.user.screen_name, 'location': location, 'has_location': has_location, 'created': tweet.user.created_at.strftime("%Y-%m-%d"), 'is_verified': int(tweet.user.verified), 'total_tweets': tweet.user.statuses_count, 'total_following': tweet.user.friends_count, 'total_followers': tweet.user.followers_count, 'total_likes': tweet.user.favourites_count, 'has_avatar': has_avatar, 'has_background': has_background, 'is_protected': int(tweet.user.protected), 'profile_modified': profile_modified}), ignore_index=True)

        return True
    
    def get_data(self):
        return self.df, self.userData
    
    def on_error(self, status_code):
        if status_code == 420:
            print("420 Error")
            return False #returning False in on_data disconnects the stream. Rather sleep

In [127]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

In [128]:
listener = MyStreamListener()
myStream = Stream(auth = auth, listener=listener)

In [129]:
myStream.filter(track=['bitcoin'])

KeyboardInterrupt: 

In [130]:
df, userData = listener.get_data()

In [136]:
df[df['User'] == 'warproxxx']

,ID,Tweet,Time,User,Likes,Replies,Retweets,in_response_to,response_type,debug
25,1009355494518132736,#bitcoin Bitcoin https://t.co/mA23cP88b8,2018-06-20 08:41:17,warproxxx,0,0,0,1009353099859972096,quoted_retweet,"Status(possibly_sensitive=False, source='Twitt..."
